In [101]:
import pandas as pd
def read_data(s, delete_name):
    '''
    s, <string>, which dataset to access
    delete_name, <set>, which features do not take into account
    '''
    if s=='test':
        df = pd.read_csv('pro_data_test.csv', delimiter=',')
    elif s=='train':
        df = pd.read_csv('pro_data_train.csv', delimiter=',')
    else:
        return
    columns = df.columns
    for column in columns:
        if column in delete_name:
            del df[column]
    #print(df.head())
    feature_len = len(df.columns)-1
    feature = df[df.columns[0:feature_len-1]]
    label = df[df.columns[feature_len:feature_len+1]]
    X = feature.values
    Y_pre = label.values
    Y = np.zeros((len(Y_pre), 2))
    for j in range(len(Y_pre)):
        if Y_pre[j] == [0]:
            Y[j][0] = 1
            Y[j][1] = 0
        else:
            Y[j][0] = 0
            Y[j][1] = 1
    return X, Y
delete_name = {'fnlwgt', 'capital-gain', 'capital-loss'}
X_train, Y_train = read_data('train', delete_name)
X_test, Y_test = read_data('test', delete_name)

In [120]:
import torch
from IPython.core.debugger import set_trace
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 70)
        #self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(70, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [121]:
net = Net()
#params = list(net.parameters())


input = torch.randn(1, 10)
print(type(input))
print(input)
input = torch.from_numpy(np.array(X_train[0])).float()
input = input.view(1, -1)
print(type(input))
print(input[0])
out = net(input)
print(out)

#input = torch.from_numpy(X_train[0])
#print(input)
#out = net(input)
#print(out)

<class 'torch.Tensor'>
tensor([[-0.2715, -1.7159, -0.0829,  0.7199,  0.0963, -0.9239, -1.5248,  0.4762,
         -0.5726, -0.6019]])
<class 'torch.Tensor'>
tensor([59.,  1.,  4.,  9.,  1.,  2.,  3.,  1.,  2., 40.])
tensor([[0.8843, 0.1157]], grad_fn=<SoftmaxBackward>)


In [122]:
output = net(input)
target = torch.from_numpy(np.array(Y_train[0])).float()  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7821, grad_fn=<MseLossBackward>)


In [123]:
import torch.optim as optim

# create your optimizer
#optimizer = optim.SGD(net.parameters(), lr=0.01)
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
# create criterion
criterion = nn.MSELoss()

# number of epochs
num_epochs = 10

total_step = len(Y_train)

# in your training loop:
for epoch in range(num_epochs):
    sf = np.arange(total_step)
    #print(sf)
    np.random.shuffle(sf)
    #print(sf)
    #print(X_train)
    X_train, Y_train = X_train[sf], Y_train[sf]
    #print(X_train)
    for i, (x, y) in enumerate(zip(X_train, Y_train)):
        input = torch.from_numpy(x).float()
        input = input.view(1, -1)
        target = torch.from_numpy(y).float()
        target = target.view(1, -1)
        
        output = net(input)
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [10000/30162], Loss: 0.0393
Epoch [1/10], Step [20000/30162], Loss: 0.0003
Epoch [1/10], Step [30000/30162], Loss: 0.5566
Epoch [2/10], Step [10000/30162], Loss: 0.1074
Epoch [2/10], Step [20000/30162], Loss: 0.1674
Epoch [2/10], Step [30000/30162], Loss: 0.0063
Epoch [3/10], Step [10000/30162], Loss: 0.0006
Epoch [3/10], Step [20000/30162], Loss: 0.0337
Epoch [3/10], Step [30000/30162], Loss: 0.2366
Epoch [4/10], Step [10000/30162], Loss: 0.2021
Epoch [4/10], Step [20000/30162], Loss: 0.0002
Epoch [4/10], Step [30000/30162], Loss: 0.4544
Epoch [5/10], Step [10000/30162], Loss: 0.0001
Epoch [5/10], Step [20000/30162], Loss: 0.0001
Epoch [5/10], Step [30000/30162], Loss: 0.0559
Epoch [6/10], Step [10000/30162], Loss: 0.0003
Epoch [6/10], Step [20000/30162], Loss: 0.0028
Epoch [6/10], Step [30000/30162], Loss: 0.0076
Epoch [7/10], Step [10000/30162], Loss: 0.0044
Epoch [7/10], Step [20000/30162], Loss: 0.4848
Epoch [7/10], Step [30000/30162], Loss: 0.0829
Epoch [8/10],

In [124]:
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in zip(X_test, Y_test):
        input = torch.from_numpy(x).float()
        input = input.view(1, -1)
        if y[0]==1:
            target = torch.tensor([0])
        else:
            target = torch.tensor([1])
        
        output = net(input)
        _, predicted = torch.max(output.data, 1)
        total += 1
        #print(predicted, target, y)
        correct += (predicted == target).sum().item()
        
print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 81.18857901726427 %
